# Unscramble

Unscramble wykorzystuje **SharedPreferences**, aby zapisywać i odczytywać stan gry. Jest to mechanizm do przechowywania danych.

Aplikacja została zaimplementowana zgodnie z wzorcem **MVVM (Model-View-ViewModel)** z wykorzystaniem **repozytorium**. Wzorzec ten pomaga oddzielić logikę biznesową od warstwy prezentacji, co ułatwia testowanie, utrzymanie i rozwijanie aplikacji.

Do zarządzania stanem aplikacji i reaktywności został użyty **LiveData**, umożliwiając łatwą aktualizację widoków w czasie rzeczywistym w odpowiedzi na zmiany w stanie gry.

Aplikacja bazuje na oficjalnym [Android Codelab](https://developer.android.com/codelabs/basic-android-kotlin-training-viewmodel#0)

<table><tr><td><img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExYWwxOGlnNGQ3Mm9xOGRraHFuaXFyOXA0dXkwNWI2OGx0M2RvbndlMSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/c4MVddvPAaRxrwp9JB/giphy.gif" width="200" /></td><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExMnVzODY5dDR1a2QxdDk0MmtibWhmZXM0ZW0yc2VkcWZwd3NkaHlmZyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/t3ERZ9ZxBVMeNiY1KY/giphy.gif" width="200" /></td><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExbW8yN201Y3EzOHN6MGZsenhlMTM1ZHU0dWY5bmtkd2lvcTVybnZyZyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/bCYIyjvwkKXnvrGd3n/giphy.gif" width="200" /></td></tr></table>

- Aplikacja zaimplementowana zgodnie ze wzoccem MVVM wraz z repozytorium
- `LiveData` został wykorzystany do łatwego zarządzania bieżącym stanem interfejsu użytkownika
- Klasa `GameUiState` modeluje stan interfejsu użytkownika
- Klasa `GameState` modeluje stan samej gry
- `SharedPreferences` został wykorzystany do zapisu i odczyta stanu gry
- Repozytorium stanowi warstwę pomiędzy `GameStateSharedPreferences` a `GameViewModel`, zapewniając izolację odpowiedzialności i umożliwiając łatwy dostęp do operacji na pliku `DataStore`


Dodajmy wymagane zależności do projektu

```kotlin
    // ViewModel
    implementation ("androidx.lifecycle:lifecycle-viewmodel:2.5.1")
    // LiveData
    implementation ("androidx.lifecycle:lifecycle-livedata:2.5.1")
```

Aplikacja będzie pozwalać zgadywać słowa, więc potrzebujemy listę słów do użycia.

In [ ]:
public final class DataProvider {

    public static final int MAX_NO_OF_WORDS = 10;
    public static final int SCORE_INCREASE = 20;
    private DataProvider(){}
    public static final List<String> words = Arrays.asList(
            "kot", "pies", "drab", "kawa", "samolot", "wino", "gracz", "woda", "dom",
            "karma", "tanie", "bilet", "muzyk", "rybak", "chleb", "motyw",
            "las", "papuga", "talerz", "stacja", "grupa", "butelka", "kurczak",
            "okno", "drzewo", "lampa", "sklep", "kasa", "broda", "papier", "szafa",
            "dzwon", "kotek"
    );
}

Gra będzie składała się z próby odgadnięcia 10 słów, za każde poprawne słowo gracz otrzymuje 20 punktów. W kolejnym kroku przygotujmy klasę `GameUiState` zawierającą wszystkie informacje, które chcemy pokazać użytkownikowi.

In [ ]:
public class GameUiState {
    private String currentScrambledWord;
    private int currentWordCount;
    private int score;
    private boolean isGuessedWordWrong;
    private boolean isGameOver;

    public GameUiState(String currentScrambledWord, int currentWordCount, int score,
                       boolean isGuessedWordWrong, boolean isGameOver) {
        this.currentScrambledWord = currentScrambledWord;
        this.currentWordCount = currentWordCount;
        this.score = score;
        this.isGuessedWordWrong = isGuessedWordWrong;
        this.isGameOver = isGameOver;
    }

    // Getters
    public String getCurrentScrambledWord() {
        return currentScrambledWord;
    }

    public int getCurrentWordCount() {
        return currentWordCount;
    }

    public int getScore() {
        return score;
    }

    public boolean isGuessedWordWrong() {
        return isGuessedWordWrong;
    }

    public boolean isGameOver() {
        return isGameOver;
    }

    // Setters
    public void setCurrentScrambledWord(String currentScrambledWord) {
        this.currentScrambledWord = currentScrambledWord;
    }

    public void setCurrentWordCount(int currentWordCount) {
        this.currentWordCount = currentWordCount;
    }

    public void setScore(int score) {
        this.score = score;
    }

    public void setGuessedWordWrong(boolean guessedWordWrong) {
        isGuessedWordWrong = guessedWordWrong;
    }

    public void setGameOver(boolean gameOver) {
        isGameOver = gameOver;
    }
}

- `currentScrambledWord: String` - aktualnie wyświetlane słowo
- `currentWordCount: Int` - licznik słów
- `score: Int` - wynik
- `isGuessedWordWrong: Boolean` - flaga pomocnicza określająca czy użytkownik poprawni odggadł słowo
- `isGameOver: Boolean` - flaga pomocnicza określająca czy gra powinna zostać zakończona

### ViewModel

Dodajmy klasę `GameViewModel`, klasa będzie zawierała kilka metod i pól:

- `LiveData<GameUiState> uiState` Jest to obiekt reprezentujący bieżący stan interfejsu użytkownika (UI). Składa się z obiektu typu `GameUiState`, który zawiera informacje o stanie gry na poziomie interfejsu użytkownika.
- `userGuess` Jest to zmienna, która przechowuje aktualne odgadnięte słowo wprowadzone przez użytkownika.
- `HashSet<String> usedWords: MutableSet<String>` Jest to zbiór słów użytych w grze. Jest to wykorzystywane w celu śledzenia, które słowa już zostały wykorzystane.
- `String currentWord` Zmienna przechowująca aktualne słowo, które jest obecnie odgrywane w grze.
- `void saveGame()` Funkcja służąca do zapisywania stanu gry. Wywołuje funkcję `saveGameState` z repozytorium, aby zapisać bieżący stan` GameUiState`, `usedWords` oraz `currentWord`.
- `void loadGame()` Funkcja służąca do wczytywania stanu gry.
- `void resetGame()` Funkcja inicjująca grę od nowa. Czyści zbiór `usedWords` i ustawia nowe, losowe słowo jako bieżące słowo w `uiState`.
- `void updateUserGuess(String guessedWord)` Funkcja aktualizująca odgadnięte słowo użytkownika (userGuess) na podstawie wprowadzonego przez niego słowa.
- `void checkUserGuess()` Funkcja sprawdzająca, czy odgadnięte słowo użytkownika (`userGuess`) jest poprawne. W zależności od wyniku, funkcja zwiększa wynik (`score`) i przygotowuje grę na kolejną rundę.
- `void skipWord()` Funkcja pozwalająca na pominięcie obecnego słowa i przechodzenie do następnego słowa.
- `void updateGameState(int updatedScore)` Funkcja, która aktualizuje stan gry (`GameUiState`). Decyduje, czy gra poinna zostać zakończona, czy też powinna przejść do kolejnej rundy.
- `String shuffleCurrentWord(String word)` Funkcja przetasowująca litery w podanym słowie i zwracająca przetasowane słowo jako wynik.
- `String pickRandomWordAndShuffle()` Funkcja wybierająca losowe słowo z dostępnych słów w `DataProvider` i przetasowująca je za pomocą `shuffleCurrentWord()`. Funkcja zapewnia, że wybrane słowo nie zostało użyte wcześniej w grze, korzystając z zbioru `usedWords`.

W pierwszym kroku dodajemy instancję `GameUiState`

In [ ]:
public LiveData<GameUiState> uiState = _uiState;

Następnie musimy sledzić aktualnie wpisane słowo przez użytkownika.

In [ ]:
private String userGuess = "";

Potrzebujemy jeszcze dwie zmienne.

In [ ]:
// Set of words used in the game
private HashSet<String> usedWords = new HashSet<>();
private String currentWord;

`usedWords` śledzące wszystkie słowa, które zostały już wylosowane z listy dostępnej w `DataProvider`, oraz `currentWord` przechowującą aktualnie wylosowane słowo.

Dodajmy funkcję resetującą całą grę.

In [ ]:
public void resetGame() {
    usedWords.clear();
    _uiState.setValue(new GameUiState(
            pickRandomWordAndShuffle(),
            1,
            0,
            false,
            false
            ));
}

- `usedWords.clear()` - czyści zbiór wykorzystanych słów
- `_uiState.setValue(...)` - inicjuje `_uiState` domyślnymi wartościami, oraz losuje aktualne słowo.

Wywołajmy funkcję `reset` w konstruktorze, inicjując w ten sposób grę.

In [ ]:
public GameViewModel() {
    super();
    resetGame();
}

Dodajmy funkcję `pickRandomWordAndShuffle()`, która losowo wybiera słowo z dostępnej listy oraz losowo przestawia jego litery.

In [ ]:
private String pickRandomWordAndShuffle() {
    // Continue picking up a new random word until you get one that hasn't been used before
    currentWord = DataProvider.words.get((int) (Math.random() * DataProvider.words.size()));
    while (usedWords.contains(currentWord)) {
        currentWord = DataProvider.words.get((int) (Math.random() * DataProvider.words.size()));
    }
    usedWords.add(currentWord);
    return shuffleCurrentWord(currentWord);
}

Wybieramy nowe słowo, dopóki funkcja nie zwróci wcześniej nieużytego. Po wyborze dodajemy nowe słowo do zbioru już użytych i wykonujemy wymieszanie liter.

In [ ]:
private String shuffleCurrentWord(String word) {
    char[] tempWord = word.toCharArray();
    // Scramble the word
    shuffleArray(tempWord);
    while (String.valueOf(tempWord).equals(word)) {
        shuffleArray(tempWord);
    }
    return String.valueOf(tempWord);
}

private void shuffleArray(char[] arr) {
    for (int i = arr.length - 1; i > 0; i--) {
        int index = (int) (Math.random() * (i + 1));
        char temp = arr[index];
        arr[index] = arr[i];
        arr[i] = temp;
    }
}

Na początku, podane słowo word jest zamieniane na tablicę znaków za pomocą metody `toCharArray()`. Dzięki temu możemy łatwo manipulować poszczególnymi literami słowa. Następnie wywołujemy funkcję `shuffleArray()` na tablicy. Metoda losowo przestawia elementy tablicy. Pętla `while` jest użyta, aby zapewnić, że przetasowanie słowa będzie różne od początkowego słowa.

W kolejnym kroku zaimplementujmy prywatną funkcję `updateGameState`, która jest odpowiedzialna za aktualizację stanu gry. Funkcja ta decyduje, czy gra się zakończyła, czy powinna przejść do kolejnej rundy.

In [ ]:
/*
 * Picks a new currentWord and currentScrambledWord and updates UiState according to
 * current game state.
 */
private void updateGameState(int updatedScore) {
    if (usedWords.size() == MAX_NO_OF_WORDS) {
        // Last round in the game, update isGameOver to true, don't pick a new word
        GameUiState currentState = _uiState.getValue();
        if (currentState != null) {
            _uiState.setValue(new GameUiState(
                    currentState.getCurrentScrambledWord(),
                    currentState.getCurrentWordCount(),
                    updatedScore,
                    false,
                    true
            ));
        }
    } else {
        // Normal round in the game
        GameUiState currentState = _uiState.getValue();
        if (currentState != null) {
            _uiState.setValue(new GameUiState(
                    pickRandomWordAndShuffle(),
                    currentState.getCurrentWordCount() + 1,
                    updatedScore,
                    false,
                    false
            ));
        }
    }
}

Wpierw sprawdzamy, czy liczba użytych słów jest równa maksymalnej liczbie słów dozwolonej w grze. Jeśli ten warunek jest spełniony, oznacza to, że gra osiągnęła ostatnią rundę. 

Następnie zaimplementujmy metodę aktualizującą słowo wprowadzone przez użytkownika

In [ ]:
public void updateUserGuess(String guessedWord) {
        userGuess = guessedWord;
}

oraz metodę odpowiedzialną za sprawdzenie, czy słwowo wprowadzone przez użytkownika jest prawidłowe

In [ ]:
public void checkUserGuess() {
    if (userGuess.equalsIgnoreCase(currentWord)) {
        if (_uiState.getValue() != null) {
            int updatedScore = _uiState.getValue().getScore() + SCORE_INCREASE;
            updateGameState(updatedScore);
        }
    } else {
        // User's guess is wrong, show an error
        GameUiState currentState = _uiState.getValue();
        if (currentState != null) {
            _uiState.setValue(new GameUiState(
                    currentState.getCurrentScrambledWord(),
                    currentState.getCurrentWordCount(),
                    currentState.getScore(),
                    true,
                    currentState.isGameOver()
            ));
        }
    }
    // Reset user guess
    updateUserGuess("");
}

Jeżeli użytkownik podał prawidłowe słowo, aktualizujemy wynik, oraz wywołujemy funkcję `updateGameState` aby przejść do kolejnej rundy. Jeżeli wprowadzone słowo jest nieprawidłowe, zmieniamy flagę `isGuessedWordWrong` na `true`. Na konie resetujemy zmienną `userGuess`.

### Zapis do pliku

Pozostało dodanie funkcjonalności zapisu i odczytu z pliku. Rozpocznijmy od dodania klasy `GameState`, która przechowuje wszystkie niezbędne informacje o stanie gry.

In [ ]:
public class GameState {
    private GameUiState gameUiState;
    private Set<String> usedWords;
    private String currentWord;

    public GameState(GameUiState gameUiState, Set<String> usedWords, String currentWord) {
        this.gameUiState = gameUiState;
        this.usedWords = usedWords;
        this.currentWord = currentWord;
    }

    // Getters
    public GameUiState getGameUiState() {
        return gameUiState;
    }

    public Set<String> getUsedWords() {
        return usedWords;
    }

    public String getCurrentWord() {
        return currentWord;
    }

    // Setters
    public void setGameUiState(GameUiState gameUiState) {
        this.gameUiState = gameUiState;
    }

    public void setUsedWords(Set<String> usedWords) {
        this.usedWords = usedWords;
    }

    public void setCurrentWord(String currentWord) {
        this.currentWord = currentWord;
    }
}

Następnie dodajmy klasę pomocniczą `GameStateDataStore`

In [ ]:
public class GameStateSharedPref {

    private static final String FILE_NAME = "game_state_shared_pref";

    private static GameStateSharedPref instance;
    private final SharedPreferences sharedPref;

    private final String CURRENT_SCRAMBLED_WORD_KEY = "current_scrambled_word";
    private final String CURRENT_WORD_COUNT_KEY = "current_word_count";
    private final String SCORE_KEY = "score";
    private final String IS_GUESSED_WORD_WRONG_KEY = "is_guessed_word_wrong";
    private final String IS_GAME_OVER_KEY = "is_game_over";
    private final String USED_WORDS_KEY = "used_words";
    private final String CURRENT_WORD_KEY = "current_word";

    private GameStateSharedPref(Application application) {
        sharedPref = application.getSharedPreferences(FILE_NAME, Context.MODE_PRIVATE);
    }

    public static synchronized GameStateSharedPref getInstance(Application application) {
        if (instance == null) {
            instance = new GameStateSharedPref(application);
        }
        return instance;
    }

    public void saveGameState(GameState gameState) {
        SharedPreferences.Editor editor = sharedPref.edit();
        editor.putString(CURRENT_SCRAMBLED_WORD_KEY, gameState.getGameUiState().getCurrentScrambledWord());
        editor.putInt(CURRENT_WORD_COUNT_KEY, gameState.getGameUiState().getCurrentWordCount());
        editor.putInt(SCORE_KEY, gameState.getGameUiState().getScore());
        editor.putBoolean(IS_GUESSED_WORD_WRONG_KEY, gameState.getGameUiState().isGuessedWordWrong());
        editor.putBoolean(IS_GAME_OVER_KEY, gameState.getGameUiState().isGameOver());
        editor.putStringSet(USED_WORDS_KEY, new HashSet<>(gameState.getUsedWords()));
        editor.putString(CURRENT_WORD_KEY, gameState.getCurrentWord());
        editor.apply();
    }

    public LiveData<GameState> loadGameState() {
        MutableLiveData<GameState> gameStateLiveData = new MutableLiveData<>();

        String currentScrambledWord = sharedPref.getString(CURRENT_SCRAMBLED_WORD_KEY, "");
        int currentWordCount = sharedPref.getInt(CURRENT_WORD_COUNT_KEY, 1);
        int score = sharedPref.getInt(SCORE_KEY, 0);
        boolean isGuessedWordWrong = sharedPref.getBoolean(IS_GUESSED_WORD_WRONG_KEY, false);
        boolean isGameOver = sharedPref.getBoolean(IS_GAME_OVER_KEY, false);
        Set<String> usedWordsSet = sharedPref.getStringSet(USED_WORDS_KEY, new HashSet<>());
        String currentWord = sharedPref.getString(CURRENT_WORD_KEY, "");

        GameUiState gameUiState = new GameUiState(
                currentScrambledWord,
                currentWordCount,
                score,
                isGuessedWordWrong,
                isGameOver
        );

        GameState gameState = new GameState(gameUiState, usedWordsSet, currentWord);
        gameStateLiveData.setValue(gameState);
        return gameStateLiveData;
    }
}

- `private final String ..._KEY` Definicje kluczy, które będą używane do przechowywania poszczególnych elementów stanu gry w `SharedPreferences`.

Dodajmy repozytorium.

In [ ]:
public class GameRepository {

    private final Application application;

    public GameRepository(Application application) {
        this.application = application;
    }

    public LiveData<GameState> loadGameState() {
        return GameStateSharedPref.getInstance(application).loadGameState();
    }

    public void saveGameState(GameState state) {
        GameStateSharedPref.getInstance(application).saveGameState(state);
    }
}


Ponieważ `SharedPreferences` wymaga kontekstu, musimy wykorzystać `AndroidViewModel`, tak jak w poprzednich przykładach.

Powróćmy do klasy `GameViewModel` i dodajmy metody odpowiedzialne za zapis i odczyt stanu gry.

In [ ]:
public void saveGame() {
    GameState gameState = new GameState(
            _uiState.getValue(),
            usedWords,
            currentWord
    );
    repository.saveGameState(gameState);
}

public void loadGame() {
    GameState gameState = repository.loadGameState().getValue();
    if (gameState != null) {
        _uiState.setValue(new GameUiState(
                gameState.getGameUiState().getCurrentScrambledWord(),
                gameState.getGameUiState().getCurrentWordCount(),
                gameState.getGameUiState().getScore(),
                gameState.getGameUiState().isGuessedWordWrong(),
                gameState.getGameUiState().isGameOver()
        ));
        usedWords = new HashSet<>(gameState.getUsedWords());
        currentWord = gameState.getCurrentWord();
    }
}

Pełny kod `GameViewModel`

In [ ]:
public class GameViewModel extends AndroidViewModel {

    private final GameRepository repository;

    // Game UI state
    private final MutableLiveData<GameUiState> _uiState = new MutableLiveData<>();
    public LiveData<GameUiState> uiState = _uiState;

    private String userGuess = "";

    // Set of words used in the game
    private HashSet<String> usedWords = new HashSet<>();
    private String currentWord;

    public GameViewModel(@NonNull Application application) {
        super(application);
        repository = new GameRepository(application);
        resetGame();
    }

    public void saveGame() {

        GameState gameState = new GameState(
                _uiState.getValue(),
                usedWords,
                currentWord
        );
        repository.saveGameState(gameState);
    }

    public void loadGame() {
        GameState gameState = repository.loadGameState().getValue();
        if (gameState != null) {
            _uiState.setValue(new GameUiState(
                    gameState.getGameUiState().getCurrentScrambledWord(),
                    gameState.getGameUiState().getCurrentWordCount(),
                    gameState.getGameUiState().getScore(),
                    gameState.getGameUiState().isGuessedWordWrong(),
                    gameState.getGameUiState().isGameOver()
            ));
            usedWords = new HashSet<>(gameState.getUsedWords());
            currentWord = gameState.getCurrentWord();
        }
    }

    /*
     * Re-initializes the game data to restart the game.
     */
    public void resetGame() {
        usedWords.clear();
        _uiState.setValue(new GameUiState(
                pickRandomWordAndShuffle(),
                1,
                0,
                false,
                false
                ));
    }

    /*
     * Update the user's guess
     */
    public void updateUserGuess(String guessedWord) {
        userGuess = guessedWord;
    }

    /*
     * Checks if the user's guess is correct.
     * Increases the score accordingly.
     */
    public void checkUserGuess() {
        if (userGuess.equalsIgnoreCase(currentWord)) {
            if (_uiState.getValue() != null) {
                int updatedScore = _uiState.getValue().getScore() + SCORE_INCREASE;
                updateGameState(updatedScore);
            }
        } else {
            // User's guess is wrong, show an error
            GameUiState currentState = _uiState.getValue();
            if (currentState != null) {
                _uiState.setValue(new GameUiState(
                        currentState.getCurrentScrambledWord(),
                        currentState.getCurrentWordCount(),
                        currentState.getScore(),
                        true,
                        currentState.isGameOver()
                ));
            }
        }
        // Reset user guess
        updateUserGuess("");
    }

    /*
     * Skip to the next word
     */
    public void skipWord() {
        if (_uiState.getValue() != null){
            updateGameState(_uiState.getValue().getScore());
            // Reset user guess
            updateUserGuess("");
        }
    }

    /*
     * Picks a new currentWord and currentScrambledWord and updates UiState according to
     * current game state.
     */
    private void updateGameState(int updatedScore) {
        if (usedWords.size() == MAX_NO_OF_WORDS) {
            // Last round in the game, update isGameOver to true, don't pick a new word
            GameUiState currentState = _uiState.getValue();
            if (currentState != null) {
                _uiState.setValue(new GameUiState(
                        currentState.getCurrentScrambledWord(),
                        currentState.getCurrentWordCount(),
                        updatedScore,
                        false,
                        true
                ));
            }
        } else {
            // Normal round in the game
            GameUiState currentState = _uiState.getValue();
            if (currentState != null) {
                _uiState.setValue(new GameUiState(
                        pickRandomWordAndShuffle(),
                        currentState.getCurrentWordCount() + 1,
                        updatedScore,
                        false,
                        false
                ));
            }
        }
    }

    private String shuffleCurrentWord(String word) {
        char[] tempWord = word.toCharArray();
        // Scramble the word
        shuffleArray(tempWord);
        while (String.valueOf(tempWord).equals(word)) {
            shuffleArray(tempWord);
        }
        return String.valueOf(tempWord);
    }

    private void shuffleArray(char[] arr) {
        for (int i = arr.length - 1; i > 0; i--) {
            int index = (int) (Math.random() * (i + 1));
            char temp = arr[index];
            arr[index] = arr[i];
            arr[i] = temp;
        }
    }

    private String pickRandomWordAndShuffle() {
        // Continue picking up a new random word until you get one that hasn't been used before
        currentWord = DataProvider.words.get((int) (Math.random() * DataProvider.words.size()));
        while (usedWords.contains(currentWord)) {
            currentWord = DataProvider.words.get((int) (Math.random() * DataProvider.words.size()));
        }
        usedWords.add(currentWord);
        return shuffleCurrentWord(currentWord);
    }
}

### Interfejs użytkownika

Ostatnim elementem będzie utworzenie fragmentu, rozpocznijmy od layotu.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<ScrollView xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent">

    <androidx.constraintlayout.widget.ConstraintLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:padding="8dp"
        tools:context=".ui.game.GameFragment">

        <Button
            android:id="@+id/skip"
            style="?attr/materialButtonOutlinedStyle"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_marginStart="8dp"
            android:layout_marginEnd="8dp"
            android:text="Pomiń"
            app:layout_constraintBaseline_toBaselineOf="@+id/submit"
            app:layout_constraintEnd_toStartOf="@+id/submit"
            app:layout_constraintStart_toStartOf="parent" />

        <Button
            android:id="@+id/submit"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_marginTop="8dp"
            android:text="Sprawdź"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toEndOf="@+id/skip"
            app:layout_constraintTop_toBottomOf="@+id/textField" />

        <Button
            android:id="@+id/save"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_margin="8dp"
            android:text="Zapisz"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/skip" />

        <Button
            android:id="@+id/load"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_margin="8dp"
            android:text="Wczytaj"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/save" />

        <TextView
            android:id="@+id/textView_unscrambled_word"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_marginTop="8dp"
            android:layout_marginBottom="8dp"
            android:text="Unscramble"
            android:textAppearance="@style/TextAppearance.MaterialComponents.Headline3"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/title"
            tools:text="Unscramble" />

        <TextView
            android:id="@+id/title"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_marginTop="20dp"
            android:layout_marginBottom="8dp"
            android:text="Unscramble"
            android:textAppearance="@style/TextAppearance.MaterialComponents.Headline4"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/word_count"
            tools:text="Unscramble" />

        <TextView
            android:id="@+id/word_count"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:text="licnzik"
            android:textAppearance="@style/TextAppearance.MaterialComponents.Headline6"
            app:layout_constraintBottom_toTopOf="@+id/title"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toTopOf="parent"
            tools:text="3 of 10 words" />

        <TextView
            android:id="@+id/score"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:text="Wynik"
            android:textAllCaps="true"
            android:textAppearance="@style/TextAppearance.MaterialComponents.Headline6"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintTop_toTopOf="parent"
            tools:text="Score: 20" />

        <com.google.android.material.textfield.TextInputLayout
            android:id="@+id/textField"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_marginTop="8dp"
            android:hint="Zgadnij"
            app:errorIconDrawable="@drawable/ic_error"
            app:helperTextTextAppearance="@style/TextAppearance.MaterialComponents.Subtitle1"
            app:layout_constraintBottom_toTopOf="@+id/submit"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/textView_unscrambled_word">

            <com.google.android.material.textfield.TextInputEditText
                android:id="@+id/userInput"
                android:layout_width="match_parent"
                android:layout_height="match_parent"
                android:inputType="textPersonName|textNoSuggestions"
                android:maxLines="1" />
        </com.google.android.material.textfield.TextInputLayout>

    </androidx.constraintlayout.widget.ConstraintLayout>
</ScrollView>

Dodajmy samą klasę

In [ ]:
public class UnscrambleFragment extends Fragment {

    private GameViewModel viewModel;

    private FragmentUnscrambleBinding binding;

    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        binding = FragmentUnscrambleBinding.inflate(inflater);

        viewModel = new ViewModelProvider(this).get(GameViewModel.class);


        // dodanie obserwatora, aktualizacja interfejsu
        viewModel.uiState.observe(getViewLifecycleOwner(), uiState -> {
            binding.wordCount.setText(uiState.getCurrentWordCount() + "/" + MAX_NO_OF_WORDS);
            binding.textViewUnscrambledWord.setText(uiState.getCurrentScrambledWord());
            binding.score.setText(String.valueOf(uiState.getScore()));

            if (uiState.isGameOver()) { // gdy gra jest zakończona pokazujemy alertDialog z podsumowaniem
                showFinalScoreDialog(uiState.getScore());
            }
        });

        // onclick przycisków
        binding.skip.setOnClickListener(v -> viewModel.skipWord());
        binding.submit.setOnClickListener(v -> viewModel.checkUserGuess());
        binding.save.setOnClickListener(v -> viewModel.saveGame());
        binding.load.setOnClickListener(v -> viewModel.loadGame());
        
        
        // odpowiedź na zmiany w polu EditText
        binding.userInput.addTextChangedListener(new TextWatcher() {
            @Override
            public void beforeTextChanged(CharSequence charSequence, int i, int i1, int i2) {

            }

            @Override
            public void onTextChanged(CharSequence charSequence, int i, int i1, int i2) {

            }

            @Override
            public void afterTextChanged(Editable editable) {
                viewModel.updateUserGuess(editable.toString()); // aktualizujemy userGuess za każdym razem, gdy użytkownik wprowadzi nowe słowo
            }
        });



        return binding.getRoot();
    }

    // funakcja definiująca AlertDialog wyświetlany po zakończeniu gry
    private void showFinalScoreDialog(int score) {
        new MaterialAlertDialogBuilder(requireContext())
                .setTitle("Gratulacje")
                .setMessage("Wynik: " + score)
                .setCancelable(false)
                .setNegativeButton("Wyjdź", (dialog, which) -> exitGame()) // po wciśnięciu aplikacja kończy działanie
                .setPositiveButton("Zagraj ponownie", (dialog, which) -> viewModel.resetGame()) // reset gry
                .show();
    }


    private void exitGame() {
        requireActivity().finish(); // wyjście z aktywności - zamknięcie aplikacji
    }
}

Możemy przetestować grę

<table><tr><td><img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExYWwxOGlnNGQ3Mm9xOGRraHFuaXFyOXA0dXkwNWI2OGx0M2RvbndlMSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/c4MVddvPAaRxrwp9JB/giphy.gif" width="200" /></td><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExMnVzODY5dDR1a2QxdDk0MmtibWhmZXM0ZW0yc2VkcWZwd3NkaHlmZyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/t3ERZ9ZxBVMeNiY1KY/giphy.gif" width="200" /></td><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExbW8yN201Y3EzOHN6MGZsenhlMTM1ZHU0dWY5bmtkd2lvcTVybnZyZyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/bCYIyjvwkKXnvrGd3n/giphy.gif" width="200" /></td></tr></table>